## Anomaly detection based on daily frequency of transactions 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split

### Loading Dataset

In [2]:
df = pd.read_csv('../SavedData/dataset2_cleaned.csv')
df['Datetime'] = pd.to_datetime(df['Datetime'])
df

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name,Datetime
0,01/01/2023,00:00:00,678330503.0,2971.000000,1584.00,NaN,Westport Care Home,2023-01-01 00:00:00
1,01/01/2023,00:00:00,472213568.0,3792.000000,1950.00,NaN,Barbiee Boutique,2023-01-01 00:00:00
2,01/01/2023,00:00:00,472213568.0,3012.000000,-780.00,283027736.0,NaN,2023-01-01 00:00:00
3,01/01/2023,00:00:00,283027736.0,1787.000000,780.00,472213568.0,NaN,2023-01-01 00:00:00
4,01/01/2023,00:00:00,624500124.0,3226.000000,1825.00,NaN,Fat Face,2023-01-01 00:00:00
...,...,...,...,...,...,...,...,...
229189,06/12/2023,20:54:00,581655972.0,45935.206861,-41.06,NaN,Tesco,2023-12-06 20:54:00
229190,06/12/2023,20:55:00,786141370.0,-244.837500,-62.35,NaN,Sainsbury Local,2023-12-06 20:55:00
229191,06/12/2023,21:05:00,824916823.0,9709.172159,-32.94,NaN,Deliveroo,2023-12-06 21:05:00
229192,06/12/2023,21:13:00,366550080.0,26834.165794,-19.25,NaN,Amazon,2023-12-06 21:13:00


### Splitting dataframe into training and testing, and expenditures and payments

In [3]:
training_transactions, testing_transactions = train_test_split(df, test_size=0.2, random_state=12)

training_payments_df = training_transactions[training_transactions['Amount'] > 0]
training_expenditures_df = training_transactions[training_transactions['Amount'] < 0]
training_expenditures_df["Amount"] = training_expenditures_df["Amount"].abs()

testing_payments_df = testing_transactions[testing_transactions['Amount'] > 0]
testing_expenditures_df = testing_transactions[testing_transactions['Amount'] < 0]
testing_expenditures_df["Amount"] = testing_expenditures_df["Amount"].abs()

/var/folders/rw/215myrns4md_n1k8nkyww6mw0000gn/T/ipykernel_2797/2449259599.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_expenditures_df["Amount"] = training_expenditures_df["Amount"].abs()
/var/folders/rw/215myrns4md_n1k8nkyww6mw0000gn/T/ipykernel_2797/2449259599.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_expenditures_df["Amount"] = testing_expenditures_df["Amount"].abs()


In [4]:
testing_expenditures_df

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name,Datetime
50365,14/03/2023,18:33:00,712474734.0,-1324.548831,31.23,NaN,Deliveroo,2023-03-14 18:33:00
50317,14/03/2023,17:28:00,419550103.0,6544.593463,125.30,NaN,Loosely Fitted,2023-03-14 17:28:00
156768,31/08/2023,23:59:00,834581602.0,925.931550,7.00,NaN,Xbox,2023-08-31 23:59:00
198076,31/10/2023,09:27:00,250957494.0,1090.427100,37.99,NaN,Sports Direct,2023-10-31 09:27:00
46751,05/03/2023,16:34:00,675231808.0,4038.532728,158.90,NaN,Selfridges,2023-03-05 16:34:00
...,...,...,...,...,...,...,...,...
177952,30/09/2023,23:59:00,887156614.0,-838.824821,5.99,NaN,Mojang Studios,2023-09-30 23:59:00
138380,31/07/2023,23:59:00,117308635.0,1863.971883,14.99,NaN,Blizzard,2023-07-31 23:59:00
27201,02/02/2023,14:46:00,299871610.0,-1106.116847,55.29,NaN,Topshop,2023-02-02 14:46:00
11255,11/01/2023,19:39:00,656812046.0,2980.102487,118.16,NaN,Tesco,2023-01-11 19:39:00


### Creating a day column 

In [5]:
training_expenditures_df['Day'] = training_expenditures_df['Datetime'].dt.date
training_payments_df['Day'] = training_payments_df['Datetime'].dt.date

/var/folders/rw/215myrns4md_n1k8nkyww6mw0000gn/T/ipykernel_2797/1200568435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_expenditures_df['Day'] = training_expenditures_df['Datetime'].dt.date
/var/folders/rw/215myrns4md_n1k8nkyww6mw0000gn/T/ipykernel_2797/1200568435.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_payments_df['Day'] = training_payments_df['Datetime'].dt.date


### Aggregating transactions by day and account

In [6]:
daily_expenditures = training_expenditures_df.groupby(['Account No', 'Day']).size().reset_index(name='Daily expenditures')
daily_payments = training_payments_df.groupby(['Account No', 'Day']).size().reset_index(name='Daily payments')

### Calculating thresholds for expenditures and payments

In [7]:
def calculate_thresholds(group, value_col):
    quantiles = group[value_col].quantile([0.25, 0.5, 0.75])
    IQR = quantiles[0.75] - quantiles[0.25]
    
    # Define the multipliers and percent increments for each threshold
    multipliers = [1.5, 2, 3]
    increments = [1, 2, 3]
    
    thresholds = []
    for multiplier, increment in zip(multipliers, increments):
        if IQR == 0:
            # When there's no variability, use the median and apply percent increment
            threshold = quantiles[0.5] * (1 + increment)
        else:
            # Standard threshold calculation using IQR
            threshold = quantiles[0.75] + multiplier * IQR
        thresholds.append(threshold)
    
    return tuple(thresholds)



In [8]:
def process_data(df, group_by_col, value_col):
    # Group by user and calculate thresholds
    thresholds = df.groupby(group_by_col).apply(lambda x: calculate_thresholds(x, value_col)).reset_index(name='Thresholds')

    # Split the tuples into separate columns
    thresholds[['Threshold 1', 'Threshold 2', 'Threshold 3']] = pd.DataFrame(thresholds['Thresholds'].tolist(), index=thresholds.index)
    
    # Drop the original 'Thresholds' column
    thresholds.drop(columns='Thresholds', inplace=True)

    return thresholds

# Example usage:
# Apply the function to daily_expenditures
daily_expenditures_thresholds = process_data(daily_expenditures, 'Account No', 'Daily expenditures')

# Apply the function to daily_payments
daily_payments_thresholds = process_data(daily_payments, 'Account No', 'Daily payments')

# Print results
print(daily_expenditures_thresholds.describe())
print(daily_payments_thresholds.describe())



         Account No  Threshold 1  Threshold 2  Threshold 3
count  9.760000e+02   976.000000   976.000000   976.000000
mean   5.508141e+08     3.620005     4.344006     5.542008
std    2.574339e+08     1.824036     2.032918     2.658760
min    1.015313e+08     1.625000     1.750000     2.000000
25%    3.313499e+08     2.000000     3.000000     4.000000
50%    5.542532e+08     3.500000     4.000000     5.000000
75%    7.668451e+08     3.500000     4.000000     5.000000
max    9.997527e+08    17.875000    21.250000    28.000000
         Account No  Threshold 1  Threshold 2  Threshold 3
count  8.750000e+02   875.000000   875.000000   875.000000
mean   5.471569e+08     2.009429     3.008000     4.008571
std    2.574037e+08     0.122473     0.152942     0.203476
min    1.015313e+08     2.000000     2.500000     3.000000
25%    3.233835e+08     2.000000     3.000000     4.000000
50%    5.506445e+08     2.000000     3.000000     4.000000
75%    7.628782e+08     2.000000     3.000000     4.0000

In [9]:
daily_expenditures_thresholds

,Account No,Threshold 1,Threshold 2,Threshold 3
0,101531259.0,5.375,6.25,8.0
1,104832000.0,1.625,1.75,2.0
2,105375973.0,7.250,8.50,11.0
3,106601471.0,2.250,2.50,3.0
4,108481285.0,3.500,4.00,5.0
...,...,...,...,...
971,995615876.0,3.500,4.00,5.0
972,996042490.0,2.000,3.00,4.0
973,998390769.0,3.500,4.00,5.0
974,998405607.0,8.500,10.00,13.0


### Risk flagging model based on frequency of transactions: Daily

In [10]:
def classify_daily_risk(testing_df, daily_thresholds_df):
    # Convert Datetime to date for accurate day-based grouping
    testing_df['Day'] = pd.to_datetime(testing_df['Datetime']).dt.date
    
    # Prepare the results list to collect data
    results = []
    
    # Iterate through each account's thresholds
    for index, row in daily_thresholds_df.iterrows():
        account_no = row['Account No']
        d1, d2, d3 = row['Threshold 1'], row['Threshold 2'], row['Threshold 3']
        
        # Filter transactions for the current account
        account_transactions = testing_df[testing_df['Account No'] == account_no]
        
        # Get unique transaction dates for the account
        unique_dates = account_transactions['Day'].unique()
        
        for transaction_date in unique_dates:
            # Filter transactions for the account on the specific date
            transactions_on_date = account_transactions[account_transactions['Day'] == transaction_date]
            
            # Count the number of transactions on the date
            total_transactions = len(transactions_on_date)
            
            # Classify risk based on transaction count compared to thresholds
            if total_transactions < d1:
                risk = 'No Risk'
            elif total_transactions < d2:
                risk = 'Low Risk'
            elif total_transactions < d3:
                risk = 'Medium Risk'
            else:
                risk = 'High Risk'
            
            # Append the result to the list
            results.append({
                'Account No': account_no,
                'Transaction Date': transaction_date,
                'Number of Transactions': total_transactions,
                'Risk': risk
            })
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

# Example usage with expenditures
results_expenditures_df = classify_daily_risk(testing_expenditures_df, daily_expenditures_thresholds)

# Example usage with payments
results_payments_df = classify_daily_risk(testing_payments_df, daily_payments_thresholds)


/var/folders/rw/215myrns4md_n1k8nkyww6mw0000gn/T/ipykernel_2797/2600554757.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_df['Day'] = pd.to_datetime(testing_df['Datetime']).dt.date
/var/folders/rw/215myrns4md_n1k8nkyww6mw0000gn/T/ipykernel_2797/2600554757.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_df['Day'] = pd.to_datetime(testing_df['Datetime']).dt.date


In [28]:
high_risk = results_expenditures_df[results_expenditures_df['Risk'] == 'High Risk']
high_risk = high_risk[high_risk['Number of Transactions'] > 7]
high_risk.head(50)

,Account No,Transaction Date,Number of Transactions,Risk
11794,414454079.0,2023-04-07,8,High Risk
16549,522188082.0,2023-06-19,9,High Risk


In [23]:
high_risk= results_expenditures_df[results_expenditures_df['Risk']== 'High Risk']
high_risk

,Account No,Transaction Date,Number of Transactions,Risk
28,104832000.0,2023-04-30,3,High Risk
30,104832000.0,2023-07-31,2,High Risk
34,104832000.0,2023-03-31,2,High Risk
38,104832000.0,2023-09-30,2,High Risk
205,108812033.0,2023-01-31,4,High Risk
...,...,...,...,...
34616,984783496.0,2023-07-31,5,High Risk
34948,987656636.0,2023-01-01,5,High Risk
35155,993039226.0,2023-03-31,4,High Risk
35210,994214354.0,2023-10-31,5,High Risk
